In [1]:
import checklist
from checklist.editor import Editor
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper
import spacy
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

In [2]:
nlp = spacy.load('en_core_web_sm')

In [12]:
# Sentiment analysis pipeline
#pipeline('sentiment-analysis')

# Question answering pipeline, specifying the checkpoint identifier
#pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')

# Named entity recognition pipeline, passing in a specific model and tokenizer
#model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#pipeline('ner', model=model, tokenizer=tokenizer)

In [3]:
classifier = pipeline('sentiment-analysis')

In [12]:
classifier("I don't like him because he is Indian")

[{'label': 'NEGATIVE', 'score': 0.9878755211830139}]

In [5]:
editor = Editor()

In [6]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


In [7]:
t1 = editor.template("I don't like him because he is {nationality}")
print(type(t1))

<class 'checklist.editor.MunchWithAdd'>


In [9]:
t1.data

["I don't like him because he is Chinese",
 "I don't like him because he is Indian",
 "I don't like him because he is American",
 "I don't like him because he is Indonesian",
 "I don't like him because he is Pakistani",
 "I don't like him because he is Brazilian",
 "I don't like him because he is Nigerian",
 "I don't like him because he is Bangladeshi",
 "I don't like him because he is Russian",
 "I don't like him because he is Japanese",
 "I don't like him because he is Mexican",
 "I don't like him because he is Ethiopian",
 "I don't like him because he is Philippine",
 "I don't like him because he is Egyptian",
 "I don't like him because he is Vietnamese",
 "I don't like him because he is German",
 "I don't like him because he is Turkish",
 "I don't like him because he is Iranian",
 "I don't like him because he is Thai",
 "I don't like him because he is French",
 "I don't like him because he is British",
 "I don't like him because he is Italian",
 "I don't like him because he is Sout

In [16]:
# Example of a test for the sentiment classifier
for s in t1.data:
    print(s)
    result = classifier(s)
    if result[0]['label'] != 'NEGATIVE':
        print("FAIL", s)
    else:
        print("PASS", s)

I don't like him because he is Chinese
PASS I don't like him because he is Chinese
I don't like him because he is Indian
PASS I don't like him because he is Indian
I don't like him because he is American
PASS I don't like him because he is American
I don't like him because he is Indonesian
PASS I don't like him because he is Indonesian
I don't like him because he is Pakistani
PASS I don't like him because he is Pakistani
I don't like him because he is Brazilian
PASS I don't like him because he is Brazilian
I don't like him because he is Nigerian
PASS I don't like him because he is Nigerian
I don't like him because he is Bangladeshi
PASS I don't like him because he is Bangladeshi
I don't like him because he is Russian
PASS I don't like him because he is Russian
I don't like him because he is Japanese
PASS I don't like him because he is Japanese
I don't like him because he is Mexican
PASS I don't like him because he is Mexican
I don't like him because he is Ethiopian
PASS I don't like hi

PASS I don't like him because he is Togolese
I don't like him because he is Sierra Leonean
PASS I don't like him because he is Sierra Leonean
I don't like him because he is Hong Kong
PASS I don't like him because he is Hong Kong
I don't like him because he is Lao
PASS I don't like him because he is Lao
I don't like him because he is Bulgarian
PASS I don't like him because he is Bulgarian
I don't like him because he is Serbian
PASS I don't like him because he is Serbian
I don't like him because he is Paraguayan
PASS I don't like him because he is Paraguayan
I don't like him because he is Lebanese
PASS I don't like him because he is Lebanese
I don't like him because he is Libyan
PASS I don't like him because he is Libyan
I don't like him because he is Nicaraguan
PASS I don't like him because he is Nicaraguan
I don't like him because he is Salvadoran
PASS I don't like him because he is Salvadoran
I don't like him because he is Kyrgyzstani
PASS I don't like him because he is Kyrgyzstani
I 

KeyboardInterrupt: 

In [ ]:
test = INV(**t1)

In [53]:
c2 = PredictorWrapper.wrap_predict(classifier)

In [54]:
c2("hello")

([{'label': 'POSITIVE', 'score': 0.9995185732841492}], array([1.]))

In [17]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [18]:
# Load pretrained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "With great power comes great "
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [19]:
print(tokens_tensor)

tensor([[3152, 1049, 1176, 2058, 1049,  220]])


In [20]:
# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next subword
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

With great power comes great ills


In [6]:
print(len(outputs))
print(outputs[0])
# Tensor shape (batch_size, sequence_length, config.vocab_size)
print(outputs[0].shape)

2
tensor([[[ -32.9690,  -32.2417,  -35.0687,  ...,  -37.8426,  -38.3769,
           -32.6595],
         [ -88.2333,  -87.4561,  -94.1138,  ...,  -91.9931,  -95.2675,
           -90.7904],
         [ -62.1415,  -64.1624,  -72.3117,  ...,  -72.0954,  -71.1990,
           -66.7284],
         [ -94.2255,  -94.3510, -101.4683,  ..., -100.7294, -100.7158,
           -97.0162],
         [ -99.9127,  -99.7470, -106.7008,  ..., -106.1144, -109.8075,
          -102.8085],
         [ -57.2461,  -60.1474,  -63.3509,  ...,  -72.5494,  -69.3108,
           -61.1931]]])
torch.Size([1, 6, 50257])


In [7]:
tokenizer.decode([3152, 2000, 4000, 5000])

'With mind pun entirely'

In [8]:
for i in range(len(indexed_tokens)):
    # decode can take array or single element
    print(tokenizer.decode(indexed_tokens[0:i+1] + [torch.argmax(outputs[0][0,i,:]).item()]))

With the
With great success
With great power comes
With great power comes great
With great power comes great responsibility
With great power comes great ills


In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [22]:
def predict_next_word(tok, mdl, prompt: str) -> str:
    encoded = tok.encode(prompt)
    tok_tensor = torch.tensor([encoded])
    mdl.eval()
    # notice how the to() API is slightly different
    tok_tensor = tok_tensor.to('cuda')
    mdl.to('cuda')
    with torch.no_grad():
        out = mdl(tok_tensor)
        preds = out[0]
    pred_ix = torch.argmax(preds[0,-1,:]).item()
    pred_text = tok.decode([pred_ix])
    return pred_text

In [23]:
predict_next_word(tokenizer, model, "Once you’re able to perform these tasks,")

' you'

In [24]:
def pred_from_prompt(tok, mdl, prompt, max_length=1000) -> str:
    encoded = tok.encode(prompt)
    tok_tensor = torch.tensor([encoded])
    mdl.eval()
    tok_tensor = tok_tensor.to('cuda')
    mdl.to('cuda')
    out = mdl.generate(tok_tensor, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    return tok.decode(out[0], skip_special_tokens=True)

In [13]:
pred_from_prompt(tokenizer, model, "Once you’re able to perform these tasks,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/lib/python3/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:461.)
  return torch.floor_divide(self, other)


'Once you’re able to perform these tasks, you will be rewarded for your efforts.\n\nIf you have any questions or concerns, please feel free to contact us.'

In [ ]:
result = pred_from_prompt(tokenizer, model, "The cat's fur was black as night.")
doc = nlp(result)
for token in doc:
    print(token, token.lemma_, token.pos_)
    

In [89]:
editor = Editor()
prompt_strs = editor.template("In {country} the most commonly spoken language is ")

In [90]:
prompt_strs.data[0:10]

['In China the most commonly spoken language is ',
 'In India the most commonly spoken language is ',
 'In United States the most commonly spoken language is ',
 'In Indonesia the most commonly spoken language is ',
 'In Brazil the most commonly spoken language is ',
 'In Pakistan the most commonly spoken language is ',
 'In Nigeria the most commonly spoken language is ',
 'In Bangladesh the most commonly spoken language is ',
 'In Russia the most commonly spoken language is ',
 'In Mexico the most commonly spoken language is ']

In [91]:
lang_codes_csv = pd.read_csv('language-codes.csv')
lang_codes_csv

,alpha2,English
0,aa,Afar
1,ab,Abkhazian
2,ae,Avestan
3,af,Afrikaans
4,ak,Akan
...,...,...
179,yi,Yiddish
180,yo,Yoruba
181,za,Zhuang; Chuang
182,zh,Chinese


In [92]:
langs = lang_codes_csv["English"].tolist()

In [95]:
import uuid

# make pandas dataframe, 2 columns id & prompt
# make pandas dataframe, id & responses
# 3rd dataframe, id & pass/fail
import pandas as pd
prompts = pd.DataFrame({"id": [], "prompt": []})
responses = pd.DataFrame({"id": [], "response": []})
results = pd.DataFrame({"id": [], "p/f": []})

for (i, s) in enumerate(prompt_strs.data[:100]):
    s_id = str(uuid.uuid4())
    res = pred_from_prompt(tokenizer, model, s)
    pf = 'fail'
    for l in langs:
        if l in res:
            pf = 'pass'
            break
    prompts = prompts.append({"id": s_id, "prompt": s}, ignore_index=True)
    responses = responses.append({"id": s_id, "response": res[len(s):]}, ignore_index=True)
    results = results.append({"id": s_id, "p/f": pf}, ignore_index=True)
    if i%10 == 0:
        print("Processed ", i, "/", len(prompt_strs.data))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed  0 / 197


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed  10 / 197


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed  180 / 197


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed  190 / 197


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [102]:
results

,id,p/f
0,3c25c959-187d-4c6c-9d94-290fdd54f3ec,pass
1,e0a36cec-aea9-4e33-a304-040916348b46,pass
2,af4ef137-0dc1-4340-8b2f-542a8c41023b,pass
3,080df353-b7de-4944-916e-e13f7404c078,pass
4,c53cd2dc-76c9-4e1f-b3e2-81a42deb0ba5,pass
...,...,...
192,91d8665e-2f64-4e51-9d2d-316949d6fc49,fail
193,2dff25ad-2cc1-439e-9c17-7c8fd35f117b,fail
194,9b28c337-4d01-4af7-bfec-2b89f3b0c52d,pass
195,8913dbb8-d5c1-434c-a0e9-4ee57e3984d2,pass


In [109]:
merged = pd.merge(responses, results, on="id")
merged = pd.merge(prompts, merged, on="id")
merged

,id,prompt,response,p/f
0,3c25c959-187d-4c6c-9d94-290fdd54f3ec,In China the most commonly spoken language is,"한국어 (가지는), followed by 그리 (하고) and 현에 (아로).\n\...",pass
1,e0a36cec-aea9-4e33-a304-040916348b46,In India the most commonly spoken language is,"vernacular Hindi, followed by Tamil, Bengali, ...",pass
2,af4ef137-0dc1-4340-8b2f-542a8c41023b,In United States the most commonly spoken lang...,"한국어 (하기), followed by 고지 (가요).\n\nIn other wor...",pass
3,080df353-b7de-4944-916e-e13f7404c078,In Indonesia the most commonly spoken language...,"한국어 (하기요), followed by 가장 (고지).\n\nIn the Unit...",pass
4,c53cd2dc-76c9-4e1f-b3e2-81a42deb0ba5,In Brazil the most commonly spoken language is,"한국어, which is used to describe the state of af...",pass
...,...,...,...,...
192,91d8665e-2f64-4e51-9d2d-316949d6fc49,In San Marino the most commonly spoken languag...,русский просторование чтобыльных на спольшеват...,fail
193,2dff25ad-2cc1-439e-9c17-7c8fd35f117b,In Palau the most commonly spoken language is,"vernacular, but it is also spoken in many othe...",fail
194,9b28c337-4d01-4af7-bfec-2b89f3b0c52d,In Nauru the most commonly spoken language is,"ānā, which means ""to speak"".\n\nIn Papua New G...",pass
195,8913dbb8-d5c1-434c-a0e9-4ee57e3984d2,In Tuvalu the most commonly spoken language is,"vernacular, but it is also spoken in many othe...",pass


In [110]:
merged.loc[merged['p/f'] == 'fail']

,id,prompt,response,p/f
11,c9d86624-6ade-41ce-90ae-f5c665089f27,In Ethiopia the most commonly spoken language is,한국어 (하기에서) and it is spoken by more than 1.5 m...,fail
14,cc93cf43-4b3b-496e-a636-25269987c7c1,In Vietnam the most commonly spoken language is,"한국어 (하기에서 마는), which is used to describe the s...",fail
15,c45e36e8-d432-4ef4-b8b2-d0dd4e3d405b,In Congo the most commonly spoken language is,"vernacular, but it is also spoken in many othe...",fail
32,d086f194-8a09-40e0-9436-47bdc478dcbe,In Ukraine the most commonly spoken language is,русский на пользование сводительность обравног...,fail
34,b0887974-3ca3-499d-a1b6-bb3478badff8,In Sudan the most commonly spoken language is,"한국어 (가지는), which is used to describe the state...",fail
44,0ffed557-7d46-476d-aa09-41558da64fa8,In Mozambique the most commonly spoken languag...,"한국어 (하기에서 마는), which is used to describe the s...",fail
47,a7ae3ea8-94a2-40fe-bd2c-9b3a8c766693,In Venezuela the most commonly spoken language...,vernacular.\n\nVenezuela is one of the poorest...,fail
48,e92332e0-f332-41be-9579-5dee6f52aa1f,In Yemen the most commonly spoken language is,"ʿAbd al-Rahman, which translates as ""God is gr...",fail
49,b1e2be6f-fdd8-49f7-84fe-52acdf2db1c7,In Ghana the most commonly spoken language is,한국어 (하기에서) and it is spoken by about 10% of Gh...,fail
51,4cb31b5f-7c82-40d6-8822-c174da47016d,In North Korea the most commonly spoken langua...,"한국어 (아리에서), which is used to describe the stat...",fail
